## Test et présentation du projet NFT

#### Importations

In [9]:
from scripts.NFT import Collection, NFT
from scripts.timestamp import now
import time

#### Créer une collection

In [10]:
ownerKey = "OwnerPublicKey_1001101"
userKey1 = "user1Key_1011001"
userKey2 = "user2Key_0100110"

# création d'une collection

collection = Collection(ownerKey, 4, 10) # remplis les champs : userPublicKey, limit et mintDuration

# autorisation de user1 ( en tant que owner)

collection.add_minter(userKey1, ownerKey)

# affichage de la collection et test

assert (userKey1 in collection.authorizedMinters) 

"Success!"


collection.display()


{'owner': 'OwnerPublicKey_1001101',
 'limit': 4,
 'mintingOpen': False,
 'mintStart': None,
 'NFTs': [],
 'blockchain': {'type': 'Blockchain',
  'blocks': [{'type': 'Block',
    'issuer': '......',
    'timestamp': 0,
    'hash': '93ae8c7bcd3208b5...',
    'signature': '...',
    'index': 0,
    'parent': '0000000000000000...',
    'certificates': [],
    'nonce': 0}]}}

#### Ouverture de la période de Minting
##### Test par utilisateur non propriétaire et utilisateur owner

In [11]:
# Utilisateur non autorisé user2 par exemple

collection.open_mint(userKey2)
# doit rendre une erreur


Exception: Pas autorisé à ouvrir le minting

In [14]:
# utilisateur autorisé (owner)

collection.open_mint(ownerKey)

assert collection.mintingOpen == True

'success'

'success'

##### Test depassement de la durée de minting

In [ ]:
# obligation d'utiliser check_mint_period (pas de tache en arrière plan en synchrone)

collection.open_mint(ownerKey)
print(collection.mintingOpen)

time.sleep(11)

collection.check_mint_period()
print(collection.mintingOpen)



True
False
